In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout,Flatten,BatchNormalization,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Data augmentation is used for generation of more images from existing images as we know that deep learning 
# is data hunry process
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Preprocess and augment the training data(standarization)
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)





/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


170498071/170498071 [==============================] - 5s 0us/step


Creating my own CNN model .

In [4]:
INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (3, 3)
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
#model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
#model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout layers
#model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=256, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [5]:
def lr_schedule(epoch):
    if epoch < 15:
        return 0.001
    if epoch < 30:
        return 0.0001
    return 0.00001

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with data augmentation and learning rate scheduling
batch_size = 100
epochs = 30
model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) / batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[LearningRateScheduler(lr_schedule)])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/30


2023-12-04 15:32:56.932544: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


500/500 [==============================] - 47s 62ms/step - loss: 2.0960 - accuracy: 0.2183 - val_loss: 2.3165 - val_accuracy: 0.2409 - lr: 0.0010
Epoch 2/30
500/500 [==============================] - 30s 60ms/step - loss: 1.7729 - accuracy: 0.3362 - val_loss: 1.9015 - val_accuracy: 0.3482 - lr: 0.0010
Epoch 3/30
500/500 [==============================] - 30s 59ms/step - loss: 1.5735 - accuracy: 0.4252 - val_loss: 1.3276 - val_accuracy: 0.5308 - lr: 0.0010
Epoch 4/30
500/500 [==============================] - 29s 59ms/step - loss: 1.3969 - accuracy: 0.5026 - val_loss: 1.4892 - val_accuracy: 0.5183 - lr: 0.0010
Epoch 5/30
500/500 [==============================] - 29s 59ms/step - loss: 1.2657 - accuracy: 0.5595 - val_loss: 1.0321 - val_accuracy: 0.6368 - lr: 0.0010
Epoch 6/30
500/500 [==============================] - 30s 59ms/step - loss: 1.1765 - accuracy: 0.5965 - val_loss: 1.1202 - val_accuracy: 0.6376 - lr: 0.0010
Epoch 7/30
500/500 [==============================] - 30s 59ms/step -

Making my CNN model more complex by adding more layers.

In [6]:
INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (3, 3)
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
#model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout layers
#model.add(Dropout(0.5))



model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [7]:
def lr_schedule(epoch):
    if epoch < 15:
        return 0.001
    if epoch < 30:
        return 0.0001
    return 0.00001

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with data augmentation and learning rate scheduling
batch_size = 100
epochs = 30
# history=model.fit(
#     datagen.flow(x_train, y_train, batch_size=batch_size),
#     steps_per_epoch=len(x_train) / batch_size,
#     epochs=epochs,
#     validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

313/313 [==============================] - 1s 3ms/step - loss: 2.3092 - accuracy: 0.0878
Test Accuracy: 8.78%


In [8]:
model6 = Sequential()
model6.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model6.add(BatchNormalization())
model6.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(Dropout(0.2))
model6.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(Dropout(0.25))
model6.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(Dropout(0.25))
model6.add(Flatten())
model6.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model6.add(BatchNormalization())
model6.add(Dropout(0.5))
model6.add(Dense(10, activation='softmax'))
# compile model
# opt = SGD(lr=0.001, momentum=0.9)
model6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
epochs = 50
history=model6.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) / batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/50


2023-12-04 15:49:05.454072: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_5/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


500/500 [==============================] - 35s 60ms/step - loss: 1.7985 - accuracy: 0.3836 - val_loss: 1.4035 - val_accuracy: 0.4998 - lr: 0.0010
Epoch 2/50
500/500 [==============================] - 30s 59ms/step - loss: 1.3361 - accuracy: 0.5225 - val_loss: 1.3899 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 3/50
500/500 [==============================] - 30s 59ms/step - loss: 1.1514 - accuracy: 0.5912 - val_loss: 1.1702 - val_accuracy: 0.6009 - lr: 0.0010
Epoch 4/50
500/500 [==============================] - 30s 59ms/step - loss: 1.0377 - accuracy: 0.6364 - val_loss: 1.0834 - val_accuracy: 0.6340 - lr: 0.0010
Epoch 5/50
500/500 [==============================] - 30s 59ms/step - loss: 0.9518 - accuracy: 0.6664 - val_loss: 0.8855 - val_accuracy: 0.6968 - lr: 0.0010
Epoch 6/50
500/500 [==============================] - 30s 59ms/step - loss: 0.8882 - accuracy: 0.6896 - val_loss: 0.7746 - val_accuracy: 0.7312 - lr: 0.0010
Epoch 7/50
500/500 [==============================] - 30s 60ms/step -